In [1]:
pip install kafka-python

                                              0.0/246.5 kB ? eta -:--:--
     ------------------                     122.9/246.5 kB 3.6 MB/s eta 0:00:01
     -------------------------------------- 246.5/246.5 kB 3.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import TopicAlreadyExistsError

# Kafka broker settings
bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers, 
    client_id='test'
)

topic_list = []
topic_list.append(NewTopic(name="example_topic", num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='example_topic', error_code=0, error_message=None)])

Createa a new topic

In [3]:
# Define topic configuration
topic_name = 'transaction_data'
partitions = 1
replication_factor = 1

# Define the topic
topic = NewTopic(name=topic_name, num_partitions=partitions, replication_factor=replication_factor)

# Create the topic
try:
    admin_client.create_topics(new_topics=[topic], validate_only=False)
    print(f"Topic '{topic_name}' created successfully.")
except TopicAlreadyExistsError:
    print(f"Topic '{topic_name}' already exists.")


Topic 'transaction_data' created successfully.


Insert Message

In [11]:
from kafka import KafkaProducer
import json
import threading

# Kafka broker settings
bootstrap_servers = 'localhost:9092'

# Kafka topic
topic_name = 'transaction_data'

# Number of messages to insert
num_messages = 1000

# Message payload
messages = [{"key": f"key_{i}", "value": f"value_{i}"} for i in range(num_messages)]

# Create Kafka producer
producer = KafkaProducer(bootstrap_servers=bootstrap_servers, value_serializer=lambda x: json.dumps(x).encode('utf-8'))

# Function to produce messages
def produce_message(message):
    producer.send(topic_name, message)

# Use threading for concurrent message production
threads = []
for message in messages:
    thread = threading.Thread(target=produce_message, args=(message,))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Flush and close the producer
producer.flush()
producer.close()
